In [35]:
from azureml.core import Workspace, Experiment, Run, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails
import json
import pandas as pd
import matplotlib.pyplot as plt
import os

In [36]:
ws = Workspace.from_config()
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print(compute.name, ":", compute.type)

nancelabCompute1 : ComputeInstance
nance8core : ComputeInstance
nanceGPU : ComputeInstance


In [40]:
#env = Environment.from_existing_conda_environment(name='training_environment', conda_environment_name='azureml_py36')
myenv = Environment(name='azureml_py36')

newenv = myenv.clone('newenv')
registered_env = newenv.register(ws)
registered_env.build_local(ws, useDocker=True)

Saving setup content into  /tmp/tmpnsl8f7he


UnsupportedOperation: fileno

In [34]:
from azureml.core import Experiment, ScriptRunConfig

# Create a script config
script_config = ScriptRunConfig(source_directory='scripts_folder',
                                script='ecm_pca_analysis.py', environment=env) 

# submit the experiment
experiment = Experiment(workspace = ws, name = 'my-experiment')
run = experiment.submit(config=script_config)
run.wait_for_completion(show_output=True)

RunId: my-experiment_1617045097_0b12ef7a
Web View: https://ml.azure.com/experiments/my-experiment/runs/my-experiment_1617045097_0b12ef7a?wsid=/subscriptions/5b7e9376-1907-45b5-b8cf-6fde28c54e67/resourcegroups/mpt_projects/workspaces/ecm_project

Streaming azureml-logs/60_control_log.txt

[2021-03-29T19:11:41.123649] Entering context manager injector.
[2021-03-29T19:11:42.567593] Using urllib.request Python 3.0 or later
Streaming log file azureml-logs/60_control_log.txt
Starting the daemon thread to refresh tokens in background for process with pid = 15121
Running: ['/bin/bash', '/tmp/azureml_runs/my-experiment_1617045097_0b12ef7a/azureml-environment-setup/conda_env_checker.sh']
Materialized conda environment not found on target: /home/azureuser/.azureml/envs/azureml_36ce7d4b564bf1da4c7a6c43b6e4dee6


[2021-03-29T19:11:42.685238] Logging experiment preparation status in history service.
Running: ['/bin/bash', '/tmp/azureml_runs/my-experiment_1617045097_0b12ef7a/azureml-environment-setup

JSONDecodeError: Expecting value: line 1 column 1 (char 0)